In [1]:
import os
import sys
import warnings; warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd

In [2]:
# добавляем в sys.path директорию со скриптами
src_dir = os.path.join(os.getcwd(), '..', 'ocp')
sys.path.append(src_dir)

In [3]:
# загружаем необходимые скрипты
from data.loading import load_data, load_obj
from data.splitting import train_holdout_split
from models.prediction import make_prediction

%load_ext autoreload
%autoreload

SEED = 26

# Загрузка данных

In [4]:
train, test, numerical, categorical = load_data('../data/processed')

In [5]:
categorical_idxs = [train.columns.get_loc(c) for c in categorical]

In [6]:
(X, _, _,
 y, _, _,
 _) = train_holdout_split(train, random_state=SEED)

In [7]:
models = load_obj('../data/models_dictionary/models_final.pkl')

In [16]:
base_models = list(models.keys())
base_models.remove('stacking')

In [17]:
base_models

['XGBClassifier', 'CatBoostClassifier']

# Предсказание

In [ ]:
# делаем предсказание на каждой 
# из построенных моделей
for model in base_models:
    pipe = models[model]['pipe']

    if model == 'CatBoostClassifier':
        # передаем в параметры обучения индексы категориальных переменных
        fit_params = {'model__cat_features': categorical_idxs,
                      'model__verbose': False}
        pipe.fit(X, y, **fit_params);
    else:
        pipe.fit(X, y)
    
    # запишем предсказанные вероятности классов под ключом test_probs
    # в ветке модели для последующего создания предсказаний для стекинга
    probs = pipe.predict_proba(test)
    models[model]['test_probs'] = probs

    # сохраняем предсказания в csv-файл
    submission = pd.DataFrame({
        'id': test.index,
        'result': probs[:, 1].flatten()
    })
    
    submission.to_csv(f'../data/models_output/submission_{model}.csv', index=False)


# создаем предсказания для стекинга с
# параметром альфа, вычисленным по отложенной выборке
alpha = models['stacking']['alpha']
xgb_probs = models['XGBClassifier']['test_probs']
cat_probs = models['CatBoostClassifier']['test_probs']
stacking_final_probs = alpha*xgb_probs + (1 - alpha)*cat_probs

submission = pd.DataFrame({
    'id': test.index,
    'result': stacking_final_probs[:, 1].flatten()
})

# сохраняем предсказания в csv-файл
submission.to_csv(f'../data/models_output/submission_stacking.csv', index=False)

In [ ]:
# делаем предсказание на каждой 
# из построенных моделей
for model in base_models:
    pipe = models[model]['pipe']

    if model == 'CatBoostClassifier':
        # передаем в параметры обучения индексы категориальных переменных
        fit_params = {'model__cat_features': categorical_idxs,
                      'model__verbose': False}
        pipe.fit(X, y, **fit_params);
    else:
        pipe.fit(X, y)
    
    # запишем предсказанные вероятности классов под ключом test_probs
    # в ветке модели для последующего создания предсказаний для стекинга
    probs = pipe.predict_proba(test)
    models[model]['test_probs'] = probs

    # сохраняем предсказания в csv-файл
    submission = pd.DataFrame({
        'id': test.index,
        'result': probs[:, 1].flatten()
    })
    
    submission.to_csv(f'../data/models_output/submission_{model}.csv', index=False)


# создаем предсказания для стекинга с
# параметром альфа, вычисленным по отложенной выборке
alpha = models['stacking']['alpha']
xgb_probs = models['XGBClassifier']['test_probs']
cat_probs = models['CatBoostClassifier']['test_probs']
stacking_final_probs = alpha*xgb_probs + (1 - alpha)*cat_probs

submission = pd.DataFrame({
    'id': test.index,
    'result': stacking_final_probs[:, 1].flatten()
})

# сохраняем предсказания в csv-файл
submission.to_csv(f'../data/models_output/submission_stacking.csv', index=False)